In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [21]:
input_columns = ['longitude', 'latitude',
       'temperature', 'humidity', 'wind_speed', 'weather_0.0',
       'weather_1.0', 'weather_2.0', 'weather_3.0', 'weather_4.0',
       'weather_5.0', 'weather_6.0', 'weather_7.0', 'weather_8.0',
       'weather_9.0', 'weather_11.0', 'weather_12.0', 'weather_13.0',
       'weather_14.0', 'weather_15.0', 'weather_16.0', 'wind_direction_0.0',
       'wind_direction_1.0', 'wind_direction_2.0', 'wind_direction_3.0',
       'wind_direction_4.0', 'wind_direction_9.0', 'wind_direction_13.0',
       'wind_direction_14.0', 'wind_direction_23.0', 'wind_direction_24.0',
       'delta_t']

output_column = "PM25_Concentration"

In [27]:
train_df = pd.read_csv("../data/beijing-18/tmp_data/train_data_mar.csv.gz")
test_df = pd.read_csv("../data/beijing-18/tmp_data/test_data_mar.csv.gz")

In [23]:
X_train_df = train_df.loc[:, input_columns]
y_train = train_df[output_column]

In [24]:
random_forest = RandomForestRegressor(n_estimators=100, criterion = "squared_error", max_depth = None)

In [25]:
random_forest.fit(X_train_df, y_train)

RandomForestRegressor()

In [28]:
X_test_df = test_df.loc[:, input_columns]
y_test = test_df[output_column]

In [30]:
y_pred = random_forest.predict(X_test_df)

In [42]:
mean_squared_error(y_test, y_pred, squared = False)

24.941738206018538

In [44]:
mean_absolute_error(y_test, y_pred)

14.116322120181785

In [46]:
r2_score(y_test, y_pred)

0.8958266209380272